# Assignments

In [6]:
import A1_XTEA, A2_XTEA_CBC

KEY : bytes = bytes.fromhex('0123456789ABCDEF0123456789ABCDEF')
DATA: bytes = 'hello world'.encode("ascii")

## xTEA

In [7]:
# get length of data
length = len(DATA)
# adjust length to fit blocksizes
adjusted_length = (int((length - 1)/ 8) + 1) * 8

# fill up remaining bytes with value 0
input_data = DATA + b'\00' * (adjusted_length - length)

encrypted_data : bytes = b''
decrypted_data : bytes = b''

for round in range(int(adjusted_length/8)):
  block = input_data[round*8:((round+1)*8)]
  encrypted_data += A1_XTEA.xtea_encrypt(key=KEY, block=block)


for round in range(int(adjusted_length/8)):
  block = encrypted_data[round*8:((round+1)*8)]
  decrypted_data += A1_XTEA.xtea_decrypt(key=KEY, block=block)

print("Success!") if decrypted_data == input_data else print("Failed!")

Success!


## xTEA + CBC

In [8]:
IV: bytes = b'deadbeef'

# get length of data
length = len(DATA)
# adjust length to fit blocksizes
adjusted_length = (int((length - 1)/ 8) + 1) * 8
block_count = int(adjusted_length/8)

# fill up remaining bytes with value 0
input_data = DATA + b'\00' * (adjusted_length - length)

encrypted_data : bytes = b''
decrypted_data : bytes = b''
old_block : bytes = b''
input_block : bytes = b''
output_block : bytes = b''

for round in range(block_count):
  block = input_data[round*8:((round+1)*8)]

  # first CBC operation
  if round == 0:
    input_block = A2_XTEA_CBC.cipher_block_chaining(block=block, old_block=b'', iv=IV, first=True)
  else:
    input_block = A2_XTEA_CBC.cipher_block_chaining(block=block, old_block=old_block, iv=IV, first=False)
  # then enciphering
  output_block = A1_XTEA.xtea_encrypt(key=KEY, block=input_block)
  encrypted_data += output_block
  old_block = output_block

for round in range(block_count):
  block = encrypted_data[(round)*8:((round+1)*8)]

  # first deciphering
  input_block = A1_XTEA.xtea_decrypt(key=KEY, block=block)

  # then CBC operation
  if round == 0:
    output_block = A2_XTEA_CBC.cipher_block_chaining(block=input_block, old_block=b'', iv=IV, first=True)
  else:
    output_block = A2_XTEA_CBC.cipher_block_chaining(block=input_block, old_block=old_block, iv=IV, first=False)
  old_block = block
  decrypted_data += output_block

print("Success!") if decrypted_data == input_data else print("Failed!")

Success!
